In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
import time
import pytz
from datetime import datetime, timezone

In [0]:
 %cd "/content/drive/My Drive/TraCAR/darknet2"

In [0]:
!make

In [0]:
### changing execute permissions for "darknet"
!chmod u+x darknet

In [0]:
### checking darknet installations
!./darknet 

# Reduce number of params, then pretrain on VOC2007

# Pretraining on VOC2007

In [0]:
%mkdir /content/VOC
%cp "/content/drive/My Drive/TraCAR/datasets/VOC/train.txt" /content/VOC/train.txt
%cp "/content/drive/My Drive/TraCAR/datasets/VOC/2007_test.txt" /content/VOC/2007_test.txt
%cp "/content/drive/My Drive/TraCAR/datasets/VOC/2007_test.txt" /content/VOC/2007_val.txt
!chmod u+x /content/VOC/train.txt
!chmod u+x /content/VOC/2007_test.txt
!chmod u+x /content/VOC/2007_val.txt

In [0]:
utc_dt = datetime.now(timezone.utc)
PST = pytz.timezone('US/Pacific')
print("\nPacific time {}\n".format(utc_dt.astimezone(PST).isoformat()))

start_time = time.time()
!./darknet detector train cfg/KLvoc.data cfg/KLyolov2-tiny-voc.cfg yolov2-tiny-voc.weights -clear 1 #not sure if need weights here

print((time.time() - start_time)/60, ' minutes elapsed' )
utc_dt = datetime.now(timezone.utc)
PST = pytz.timezone('US/Pacific')
print("\nPacific time {}\n".format(utc_dt.astimezone(PST).isoformat()))

# Test on VOC2007

In [0]:
# !./darknet detect cfg/yolov2-tiny-voc.cfg yolov2-tiny-voc.weights data/kite.jpg
!./darknet detect cfg/yolov2-tiny-voc.cfg yolov2-tiny-voc.weights ../datasets/VOC/VOCdevkit/VOC2007/JPEGImages/009962.jpg

# Test on UFPR

In [0]:
# !./darknet detect cfg/KLyolov2-tiny-UFPR.cfg yolov2-tiny-voc.weights ../datasets/UFPR-ALPR/testing/track0094/track0094\[01\].png 
!./darknet detect cfg/yolov2-tiny-voc.cfg yolov2-tiny-voc.weights ../datasets/UFPR-ALPR/testing/track0094/track0094\[01\].png 

# Finetuning on UFPR-ALPR


In [0]:
%mkdir /content/UFPR
%cp "/content/drive/My Drive/TraCAR/datasets/UFPR-ALPR/training.txt" /content/UFPR/training.txt
%cp "/content/drive/My Drive/TraCAR/datasets/UFPR-ALPR/testing.txt" /content/UFPR/testing.txt
%cp "/content/drive/My Drive/TraCAR/datasets/UFPR-ALPR/validation.txt" /content/UFPR/validation.txt
!chmod u+x /content/UFPR/training.txt
!chmod u+x /content/UFPR/testing.txt
!chmod u+x /content/UFPR/validation.txt


In [0]:
log_path = "/content/drive/My Drive/TraCAR/darknet2/KLbackup/UFPR/v13/log.txt"

utc_dt = datetime.now(timezone.utc)
PST = pytz.timezone('US/Pacific')
print("\nPacific time {}\n".format(utc_dt.astimezone(PST).isoformat()))

start_time = time.time()
# !./darknet detector train cfg/KLUFPR-ALPR.data cfg/KLyolov2-tiny-UFPR.cfg yolov2-tiny-voc.weights -clear 1
!./darknet detector train cfg/KLUFPR-ALPR.data cfg/KLyolov2-tiny-UFPR.cfg > "$log_path"
# %mv log.txt $log_path

print((time.time() - start_time)/60, ' minutes elapsed' )

utc_dt = datetime.now(timezone.utc)
PST = pytz.timezone('US/Pacific')
print("\nPacific time {}\n".format(utc_dt.astimezone(PST).isoformat()))

# Visualize loss curve

In [0]:
import matplotlib.pyplot as plt

def visualize_loss(log_path):
  file1 = open(log_path, 'r') 
  Lines = file1.readlines() 
  loss = []
  for line in Lines:
    line_fields = line.split(' ')
    try:
      loss_index_inc = line_fields.index('avg,')
      loss_val = float(line_fields[loss_index_inc - 1])
      loss.append(loss_val)
    except:
      pass 
  plt.plot(loss)
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.ylim((0,15))


visualize_loss(log_path)

# Test new model on UFPR

In [0]:
# make sure model directory is correct
# !./darknet detect cfg/KLyolov2-tiny-UFPR.cfg KLbackup/UFPR/v12/KLyolov2-tiny-UFPR_14000.weights ../datasets/UFPR-ALPR/training/track0006/track0006\[01\].png 
!./darknet detect "KLbackup/UFPR/v9/Copy of KLyolov2-tiny-UFPR.cfg" KLbackup/UFPR/v9/KLyolov2-tiny-UFPR_12000.weights ../datasets/UFPR-ALPR/training/track0006/track0006\[01\].png
# !./darknet detect cfg/KLyolov2-tiny-UFPR.cfg KLbackup/UFPR/v9/KLyolov2-tiny-UFPR_12000 .weights data/tracktest.png